Done so far :


*   Lemmatization
*   Stop Words Removal

Verify :

* Normalization - removing accents, etc.
* Dates replaced with strings



In [1]:
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
import wordninja 

####### After importing nltk, run the following only once ######
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')
### pip install wordninja ###

In [130]:
def remove_htmlcodes(document):
    
    replacement = {
                    "&ampnbsp": ' ',
                    "&ampamp": '&',
                    "&ampquot": '\'',
                    "&ampldquo": '\"',
                    "&amprdquo": '\"',
                    "&amplsquo": '\'',
                    "&amprsquo": '\'',
                    "&amphellip": '...',
                    "&ampndash": '-',
                    "&ampmdash": '-'
                  }
    for str in replacement:
        document = document.replace(str, replacement[str])
        
    return document

In [2]:
def get_wordnet_pos(word):
    tag=nltk.pos_tag([word])[0][1][0].upper()
    tag_dict={"J": wordnet.ADJ, 
              "N": wordnet.NOUN,
              "V": wordnet.VERB,
              "R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemma_stop(str):
    lemmatizer = WordNetLemmatizer()
    # sentence=arr[0][4]
    tokenizer = RegexpTokenizer('\w+|\$]\d\[+|\S+,-')
    tokenized = tokenizer.tokenize(str)
    lemmatized = [lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in tokenized]
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in lemmatized if w.lower() not in stop_words]
    after_lemma_stop = ' '.join(w for w in filtered_sentence)
    return filtered_sentence

In [3]:
# loading data.npy
# data.npy is a 2D array containing the dataset information as
# data[i][0] : docID of ith document
# data[i][1] : title of ith document
# data[i][4] : content of ith document

data = np.load('data.npy',allow_pickle = True)
# sentence = data[0][4]
# print(sentence)

In [4]:
# creating a map {index_in_data_npy, docID}

# ex. if ith element in data has docID j,
# get_docID[i] will return j

get_docID = {}
get_index = {}

print(len(data))

for i in range(0, len(data)) :
    get_docID[i] = int(data[i][0])
    get_index[int(data[i][0])] = i
    # print(get_docID[i])

204135


In [5]:
def is_not_credible (text):
    match = re.search(r'[!@#?&{}()]', text)
    
    if match:
        return True
    else:
        return False

In [6]:
def scrub_words(text):
    text = re.sub('[!@#?&{}()]', '', text)
    text=re.sub(r'[^\x00-\x7F]'," ",text)
    return text

In [7]:
def clean_document (document_string):
    cleaned_doc = document_string
    for word in document_string.split():
                if is_not_credible(word):
                    temp= scrub_words(word)
                    split=wordninja.split(temp)
                    if len(split)>7:
                          cleaned_doc = cleaned_doc.replace(word,'')
    return cleaned_doc

In [8]:
from datetime import datetime

def replace_dates(documentString):
    
    regEx = '(([0-9]+(/|\\.|-)[0-9]+(/|\\.|-)[0-9]+)|([0-9]+(/|\\.|-)[0-9]+))'
    iterator = re.finditer(regEx, documentString)
    listOfDates = [(m.start(0), m.end(0)) for m in iterator]
    
    for indices in listOfDates:
        date = documentString[indices[0]:indices[1]]
        tmp = date
        date = date.replace('.', '/')
        date = date.replace('-', '/')
        count = date.count('/')
        newDate = ''
        if count == 2:
            try:
                newDate = datetime.strptime(date, '%m/%d/%Y').strftime('%d %b %Y')
            except ValueError as ve:
                newDate = date
        else:
            try:
                newDate = datetime.strptime(date, '%m/%d').strftime('%d %b')
            except ValueError as ve:
                newDate = date
                
        newDate = newDate.replace(' ', '')
        documentString = documentString.replace(tmp, newDate)
        # print(newDate)
    
    return documentString

In [9]:
# Before cleaning 

print(len(lemma_stop(data[get_index[212853]][4])))
print(lemma_stop(data[get_index[212853]][4]))

1891
['know', 'States', 'impose', 'reasonable', 'limitation', 'less', 'gun', 'crime', 'less', 'homicide', 'Sen', 'Chris', 'Murphy', 'Conn', 'speak', 'Senate', 'floor', 'June', '15', 'Readers', 'ask', 'fact', 'check', 'gun', 'rhetoric', 'use', 'Democrats', 'wake', 'mass', 'shoot', 'Orlando', 'one', 'case', 'already', 'delve', 'material', 'claim', 'new', 'let', 'take', 'look', 'start', 'Murphy', 'statement', 'Facts', 'Murphy', 'staff', 'say', 'refer', 'chart', 'appear', 'National', 'Journal', '2015', 'turn', 'carefully', 'checked', 'chart', 'President', 'Obama', 'make', 'similar', 'carefully', 'phrase', 'claim', 'gun', 'death', 'Note', 'Murphy', 'refer', 'homicide', 'gun', 'crime', 'President', 'Obama', 'earn', 'Two', 'Pinocchios', 'Readers', 'check', 'full', 'fact', 'check', 'summary', 'note', 'gun', 'death', '60', 'percent', '2013', 'actually', 'suicide', 'data', 'use', 'National', 'Journal', 'chart', 'calculates', 'number', 'gun', 'related', 'death', 'per', '100', '000', 'people', 'in

In [10]:
# After cleaning (removing JSON, HTML, etc)

cleaned_doc=clean_document(data[get_index[212853]][4])
print(len(lemma_stop(cleaned_doc)))
print(lemma_stop(cleaned_doc))

1116
['know', 'States', 'impose', 'reasonable', 'limitation', 'less', 'gun', 'crime', 'less', 'homicide', 'Sen', 'Chris', 'Murphy', 'Conn', 'speak', 'Senate', 'floor', 'June', '15', 'Readers', 'ask', 'fact', 'check', 'gun', 'rhetoric', 'use', 'Democrats', 'wake', 'mass', 'shoot', 'Orlando', 'one', 'case', 'already', 'delve', 'material', 'claim', 'new', 'let', 'take', 'look', 'start', 'Murphy', 'statement', 'Facts', 'Murphy', 'staff', 'say', 'refer', 'chart', 'appear', 'National', 'Journal', '2015', 'turn', 'carefully', 'checked', 'chart', 'President', 'Obama', 'make', 'similar', 'carefully', 'phrase', 'claim', 'gun', 'death', 'Note', 'Murphy', 'refer', 'homicide', 'gun', 'crime', 'President', 'Obama', 'earn', 'Two', 'Pinocchios', 'Readers', 'check', 'full', 'fact', 'check', 'summary', 'note', 'gun', 'death', '60', 'percent', '2013', 'actually', 'suicide', 'data', 'use', 'National', 'Journal', 'chart', 'calculates', 'number', 'gun', 'related', 'death', 'per', '100', '000', 'people', 'in

*Run the following cell once after all pre-processing (removing JSON etc), and store final lemmatized contents of all docs:* 

In [11]:
# Run once after all pre-processing 


# # Tester code
# import time

# s = time.time()

# for i in range(0, len(data)) :
#     f_content = clean_document(data[i][4])
#     contents = f_content
#     # print(contents)
#     final = lemma_stop (contents)
# #     print(type(final))
#     # print (final)
    
# print(time.time()-s)

In [12]:
# creating a temporary smaller dataset

subset = []
counter = 0
for document in data:
    subset.append(document)
    counter += 1
    if counter == 1000:
        break

In [150]:
import time
from tqdm import tqdm

start = time.time()

titles = []
contents = []
for document in tqdm(subset):
    # actually modifying the document
    document[4] = remove_htmlcodes(document[4])
    
    # not actually modifying the document
    modifiedContent = replace_dates(document[4])
    modifiedContent = lemma_stop(modifiedContent)
    modifiedTitle = lemma_stop((document[1]))
    # modifiedContent = lemma_stop(clean_document(document[4]))
    # modifiedTitle = lemma_stop(clean_document(document[1]))
    titles.append(modifiedTitle)
    contents.append(modifiedContent)
    
print(time.time() - start)  # 110.26236414909363

100%|██████████| 1000/1000 [01:50<00:00,  9.03it/s]

110.75669765472412


In [151]:
import unidecode
contents_temp = contents
titles_temp = titles
for i in range(1000):
    for j in range(len(contents[i])):
        contents[i][j] = unidecode.unidecode(contents[i][j])
    for j in range(len(titles[i])):
        titles[i][j] = unidecode.unidecode(titles[i][j])
# for document in contents_temp:
#     for word in document:
#         word = unidecode.unidecode(word)
# for title in titles_temp:
#     for word in title:
#         word = unidecode.unidecode(word)

In [152]:
import trie

# Create map from docID of the document to an object of class Node 
# (i.e, the corresponding document trie structure)
# ex. if the docID of the document is 1, 
# getReference[1] gives the object which is the trie structure of docID 1

getReference = {}

In [153]:
documentRoot = []
collection = trie.CollectionNode()

# initializing the root for 1000 documents
for i in range(1000):
    newDocument = trie.Node()
    documentRoot.append(newDocument)
    getReference[get_docID[i]] = newDocument

In [154]:
# creating the documents

max_tf = {}

import time
from tqdm import tqdm

start = time.time()
for i in tqdm(range(1000)):
    for w in contents_temp[i]:
        collection.add_document(w, 0, get_docID[i])
        documentRoot[i].add(w, 0)
        if get_docID[i] in max_tf:
            max_tf[get_docID[i]] = max(documentRoot[i].count_words(w, 0), max_tf[get_docID[i]])
        else:
            max_tf[get_docID[i]] = documentRoot[i].count_words(w, 0)
    for w in titles_temp[i]:
        collection.add_title(w, 0, get_docID[i])
        
print(time.time() - start)  #39.19705152511597

100%|██████████| 1000/1000 [00:33<00:00, 29.64it/s]

33.7455792427063


In [155]:
import math
import queue

documentLength = {}
N = len(documentRoot)

for i in tqdm(range(len(documentRoot))):
    
    docID = get_docID[i]
    length = 0
    document = documentRoot[i]
    q = queue.Queue()
    q.put([document, ''])

    while q.qsize() > 0:

        current = q.get()
        reference = current[0]
        word = current[1]

        if reference.words > 0:
            df = len(collection.get_doc_list(word, 0))
            idf = math.log10(N/df)
            # print(word, reference.words, df)
            length += (reference.words * idf) ** 2

        for i in range(256):
            if reference.children[i] is not None:
                new_word = word + chr(i)
                q.put([reference.children[i], new_word])

    # print(length**0.5)
    documentLength[docID] = length**0.5

100%|██████████| 1000/1000 [00:40<00:00, 24.74it/s]


In [19]:
# import sys
# print(sys.getsizeof(documentRoot))

In [165]:
query = 'net neutrality'
final_query = replace_dates(query)
final_query = lemma_stop(final_query)
for i in range(len(final_query)):
    final_query[i] = unidecode.unidecode(final_query[i])
print(final_query)
print(len(final_query))

['net', 'neutrality']
2


In [166]:
tf_query = {}
for w in final_query:
    if w not in tf_query:
        tf_query[w] = 1
    else:
        tf_query[w] += 1
        
    # Test code just to see distribution of query terms in the documents
    
    # print(w)
    # df = len(collection.get_doc_list(w,0))
    # print(collection.get_doc_list(w,0))


In [167]:
# print(type(documentRoot[1]), get_docID[1])
# document = documentRoot[1]
# N = len(documentRoot)

# import queue
# import math

# length = 0
# q = queue.Queue()
# q.put([document, ''])

# while q.qsize() > 0:
    
#     current = q.get()
#     reference = current[0]
#     word = current[1]
    
#     if reference.words > 0:
#         df = len(collection.get_doc_list(word, 0))
#         idf = math.log10(N/df)
#         # print(word, reference.words, df)
#         length += (reference.words * idf) ** 2
    
#     for i in range(256):
#         if reference.children[i] is not None:
#             new_word = word + chr(i)
#             q.put([reference.children[i], new_word])

# print(length**0.5)
# print(replace_dates(subset[get_index[104]][4]))
# replace_dates('12/12')
# lemma_stop('12Dec')

***Ranked Retrieval based on TF-IDF Score :***


In [168]:
# scores[i] stores the dot product of the tf-idf score vectors of the query and document of docID i in the corpus
scores = {}
title_score = {}

# N is the total number of documents in the corpus
N = len(documentRoot)

# wordsInDoc[i] is a sorted list of (word, score) tuples where
# score is the tf-idf score for the (word, <ith doc>) pair
wordsInDoc = {}

factor = {}

import math
import bisect

for query_term in tf_query:
    
    docs_having_query_term = collection.get_doc_list(query_term, 0)
    df = len(docs_having_query_term)
    idf = 0
    
    print('-------------------------------------')
    print('Term in query = ', query_term)
    # print('List of documents with this term=', docs_having_query_term)
    print()
    
    if df == 0:
        idf = 0
    else:
        idf = math.log10(N/df)
        
    docs_having_query_term_in_title = collection.get_title_list(query_term,0)
    for docID in docs_having_query_term_in_title:
        if docID in title_score:
            title_score[docID] += math.log10(N/df)
        else:
            title_score[docID] = math.log10(N/df)
        
    print('df = ',df)
    print('idf = ',idf)
    
    tfidf_query = tf_query[query_term] * idf
        
    for docID in docs_having_query_term:
        # print(docID)
        tf_doc = getReference[docID].count_words(query_term, 0)
        tf_doc = 0.5 + 0.5*tf_doc/max_tf[docID]
        # print('tf for doc',docID,'is',tf_doc)
        # tfidf_doc_query = tf_doc * idf
        # tfidf_doc = 1 + math.log10(tf_doc)
        tfidf_doc = (tf_doc)
        # tfidf_doc_query = (tf_doc)
        
        # print('tfidf for doc',doc,'is',tfidf_doc)
        # print()
        
        if docID not in scores:
            scores[docID] = (tfidf_query * tfidf_doc)
            wordsInDoc[docID] = []
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] = idf
        else:
            scores[docID] += (tfidf_query * tfidf_doc)
            bisect.insort(wordsInDoc[docID], [-tfidf_query * tfidf_doc, query_term])
            factor[docID] += idf
print(title_score)
for docID in scores:
    if documentLength[docID] != 0:
#         scores[docID] = scores[docID]/ math.sqrt(documentLength[docID])
        scores[docID] *= factor[docID]
        if docID in title_score:
            scores[docID] *= 1 + title_score[docID]


sorted_scores = sorted(scores.items(), key = lambda kv : kv[1] , reverse = True)

maxshow = min(10, len(scores))

print('\n\n')
print('============================================')

for i in range(maxshow):
    # print(i)
    print()
    docID = sorted_scores[i][0]
    print('doc ID = ', docID)
    cnt = 0
    print('Keywords:')
    print()
    print(subset[get_index[sorted_scores[i][0]]][1])
    print()
    if sorted_scores[i][0] not in title_score:
        print('title score = ',0)
    else:
        print('title score = ',title_score[sorted_scores[i][0]])
    for j in range(len(wordsInDoc[docID])):
        print(wordsInDoc[docID][j][1], wordsInDoc[docID][j][0], end = ' ')
        print(getReference[docID].count_words(wordsInDoc[docID][j][1], 0))
    print()
    print()
    count = 0
    found = 0
    words_before=queue.Queue()
    at_start = 1
    display = ""
    for word in subset[get_index[docID]][4].split():
            
        check_with=replace_dates(word)
        if len(lemma_stop(check_with)) > 0:
            check_with=lemma_stop(check_with)[0]
        else:
            check_with=word
        
        if check_with == wordsInDoc[docID][0][1]:
            found=1
            
        if found == 1:
            display = display + word + " "
            count += 1
            if count == 50:
                break
        if found == 0:
            words_before.put(word)
            if words_before.qsize()>20:
                remove=words_before.get()
                at_start=0
                
    if not at_start:
        print('...', end = ' ')
    while words_before.qsize() > 0:
        print(words_before.get(), end = ' ')
    print(display, end = ' ')
    print('...', end = ' ')
    print('\n')
    print('tf-idf score=', sorted_scores[i][1])
    print('\n')
    print('============================================')

-------------------------------------
Term in query =  net

df =  43
idf =  1.3665315444204134
-------------------------------------
Term in query =  neutrality

df =  17
idf =  1.7695510786217261
{6913: 3.1360826230421397, 6565: 3.1360826230421397, 7018: 3.1360826230421397, 6768: 3.1360826230421397, 7216: 1.3665315444204134, 6876: 3.1360826230421397, 6846: 3.1360826230421397, 6773: 1.7695510786217261}




doc ID =  6846
Keywords:

What can Obama really do for net neutrality?

title score =  3.1360826230421397
neutrality -1.7175054586622636 16
net -1.286147335925095 15


Net neutrality might be the most contentious problem the Federal Communications Commission has faced in recent history. The past year has been marked by a landmark legal case that struck down the 2010 Open Internet rules, and a series of polarizing proposals for new ones. Protestors have driven a record number  ... 

tf-idf score= 38.96067127710208



doc ID =  6876
Keywords:

Without net neutrality, what stops HBO fro

In [169]:
check_with=replace_dates('9/11')
check_with=str(lemma_stop(check_with))
print(check_with)

TypeError: 'str' object is not callable

***Original Text :***

      We’re back in Dale Cooper’s position, wandering through a freshly revived world, and trying to catch up with the ways it’s moved on in his absence.

***Processed Text :***

      We back Dale Cooper position wander freshly revive world try catch way move absence

In [171]:
print(len(subset[get_index[6913]][4]))
print(len(subset[get_index[6773]][4]))

5079
6181


In [ ]:
# import wordninja
# print(wordninja.split('DragonSlayers'))
# str = '&nbsp;&amp;&quot;&copy;&reg;&trade;&ldquo;&rdquo;&lsquo;&rsquo;&laquo;&raquo;&lsaquo;&rsaquo;&sect;&para;&bull;&middot;&hellip;&brvbar;&ndash;&mdash;'
# str = str.replace(';', ' ')
# str = str.replace('&', '&amp')
# print(str)

In [149]:
test = 'nothing else matters.&amprdquo'
# print(test)
# print(test.replace('&ampnbsp',' '))

replacement =   {
                    "&ampnbsp": ' ',
                    "&ampamp": '&',
                    "&ampquot": '\'',
                    "&ampldquo": '\"',
                    "&amprdquo": '\"',
                    "&amplsquo": '\'',
                    "&amprsquo": '\'',
                    "&amphellip": '...',
                    "&ampndash": '-',
                    "&ampmdash": '-'
                }
for str in replacement:
    test = test.replace(str, replacement[str])

print(test)

nothing else matters."
